In [7]:
import os
from langchain_together import ChatTogether
from langgraph.graph import StateGraph, END
from tavily import TavilyClient
from langchain_core.messages import HumanMessage, SystemMessage


os.environ["TAVILY_API_KEY"] = "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"
os.environ["TOGETHER_API_KEY"] = "cc9d912a47b3e176be7b130a25fa6872088da3c3b75340b77091153fe2acb78b"

tavily_client = TavilyClient()

def search_tavily(query: str):
    result = tavily_client.search(query=query, search_depth="advanced", max_results=5)
    return result['results']

llm = ChatTogether(
    model="mistralai/Mixtral-8x7B-Instruct-v0.1",  # Good quality open-source model
    temperature=0.2,
    max_tokens=1024,
)

#AGENT 1
def researcher_node(state):
    query = state['query']
    links = search_tavily(query)

    documents = "\n".join([f"Title: {link['title']}\nURL: {link['url']}\nSnippet: {link['content']}" for link in links])
    return {"query": query, "documents": documents}

#AGENT 2:
def answer_drafter_node(state):
    query = state['query']
    documents = state['documents']

    response = llm.invoke([
        SystemMessage(content="You are a helpful research assistant. Use the documents below to answer the question clearly."),
        HumanMessage(content=f"Question: {query}\n\nDocuments:\n{documents}")
    ])

    return {
        "query": query,
        "documents": documents,
        "answer": response.content
    }
workflow = StateGraph(dict)
workflow.add_node("research_agent", researcher_node)
workflow.add_node("answer_drafter", answer_drafter_node)
workflow.set_entry_point("research_agent")
workflow.add_edge("research_agent", "answer_drafter")
workflow.add_edge("answer_drafter", END)
workflow = workflow.compile()


def run_deep_research_agentic_system(user_query):
    inputs = {"query": user_query}
    result = workflow.invoke(inputs)
    return result["answer"]

query = "What are the upcoming updates of LLM?"
final_answer = run_deep_research_agentic_system(query)
print("\n--- FINAL ANSWER ---\n")
print(final_answer)



--- FINAL ANSWER ---

 Based on the documents provided, the following upcoming updates for Large Language Models (LLMs) in 2025 are:

1. GPT-4.5 by OpenAI, release date: Feb 27, 2025.
2. Claude 3.7 Sonnet by Anthropic, release date: Feb 24, 2025.
3. Grok-3 by xAI, release date: Feb 17, 2025.
4. Gemini 2.0 Flash-Lite and Gemini 2.0 Pro by Google DeepMind, release date: Feb 5, 2025.
5. GPT-o3-mini by OpenAI, release date: Jan 31, 2025.
6. Qwen 2.5-Max by Alibaba, release date: Jan 29, 2025.

In addition to these releases, there are also updates regarding the capabilities and features of LLMs:

* LLaMA 3.2 by Meta, released in September 2024, introduces multimodal capabilities for processing both text and images.
* Gemini 2.0 Pro and Gemini 2.0 Flash have an increased context length of 2,000,000 and 1,000,000 tokens, respectively.
* LLaMA 3.1 has 405 billion parameters and an expanded context length of 128,000.

Lastly, there are trends and innovations shaping the future of AI LLMs, incl